## Updated DTI Processing Pipeline using DIPY -- Run in Jupyter Notebook

In [ ]:
import os
import subprocess
from tqdm import tqdm
import numpy as np
import nibabel as nib
from dipy.denoise.nlmeans import nlmeans
from dipy.denoise.noise_estimate import estimate_sigma
from dipy.reconst.dti import TensorModel, gradient_table
from dipy.io import read_bvals_bvecs
import gzip
import shutil

os.environ['PATH'] += ':/Users/christianraimondo/ants-2.5.0/bin'
nthreads = os.environ.get('MRTRIX_NTHREADS', 'Not set')
print(f'MRTRIX_NTHREADS is {nthreads}')


def run_command(command, step_name):
    print(f"Running: {step_name}")
    result = subprocess.run(command, shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if result.returncode != 0:
        print(f"Command failed with code {result.returncode}: {result.stderr}")
    return result

def denoise_dti(output_dir, dti_path, step_name):
    print(f"Running: {step_name}")
    img = nib.load(dti_path)
    data = img.get_fdata()
    sigma = estimate_sigma(data, N=4)  # N is the number of coils. Adjust as needed
    denoised_data = nlmeans(data, sigma=sigma)
    denoised_img = nib.Nifti1Image(denoised_data, img.affine)
    denoised_dti_path = os.path.join(output_dir, "dti_denoise.nii.gz")
    nib.save(denoised_img, denoised_dti_path)
    return denoised_dti_path

def fit_tensors_with_dipy(output_dir, bvec_path, bval_path, mask_path, step_name):
    print(f"Running: {step_name}")
    denoised_dti_path = os.path.join(output_dir, "dti_denoise.nii.gz")
    dti_img = nib.load(denoised_dti_path)
    data = dti_img.get_fdata()
    affine = dti_img.affine
    
    bvals, bvecs = read_bvals_bvecs(bval_path, bvec_path)
    gtab = gradient_table(bvals, bvecs)
    mask = nib.load(mask_path).get_fdata().astype(bool)
    
    tenmodel = TensorModel(gtab)
    tenfit = tenmodel.fit(data, mask)
    
    # Save the Fractional Anisotropy (FA)
    FA_img = nib.Nifti1Image(tenfit.fa.astype(np.float32), affine)
    nib.save(FA_img, os.path.join(output_dir, "FA.nii.gz"))
    
    # Save primary, secondary, and tertiary eigenvectors (V1, V2, V3)
    for i, v in enumerate([tenfit.evecs[..., 0], tenfit.evecs[..., 1], tenfit.evecs[..., 2]]):
        nib.save(nib.Nifti1Image(v.astype(np.float32), affine), os.path.join(output_dir, f"V{i+1}.nii.gz"))
    
    # Save the Mean Diffusivity (MD), Axial Diffusivity (AD), Radial Diffusivity (RD)
    MD_img = nib.Nifti1Image(tenfit.md.astype(np.float32), affine)
    nib.save(MD_img, os.path.join(output_dir, "MD.nii.gz"))
    
    RD_img = nib.Nifti1Image(tenfit.rd.astype(np.float32), affine)
    nib.save(RD_img, os.path.join(output_dir, "RD.nii.gz"))
    
    AD_img = nib.Nifti1Image(tenfit.ad.astype(np.float32), affine)
    nib.save(AD_img, os.path.join(output_dir, "AD.nii.gz"))
    
    # Save the eigenvalues (L1, L2, L3)
    for i, l in enumerate([tenfit.evals[..., 0], tenfit.evals[..., 1], tenfit.evals[..., 2]]):
        nib.save(nib.Nifti1Image(l.astype(np.float32), affine), os.path.join(output_dir, f"L{i+1}.nii.gz"))

def ensure_gzipped_dti(input_dir):
    """Ensure the DTI file is in gzip format."""
    gzipped_dti_path = os.path.join(input_dir, "dti.nii.gz")
    if not os.path.exists(gzipped_dti_path):
        dti_path = os.path.join(input_dir, "dti.nii")
        if os.path.exists(dti_path):
            # If dti.nii exists but dti.nii.gz does not, gzip dti.nii
            with open(dti_path, 'rb') as f_in:
                with gzip.open(gzipped_dti_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
            print("dti.nii was gzipped to dti.nii.gz")
        else:
            raise FileNotFoundError("Neither dti.nii.gz nor dti.nii was found in the directory.")
    return gzipped_dti_path

def process_dti(input_dir):
    steps = ["Ensure Gzipped DTI", "Convert to MRtrix", "Denoising", "Extract b0 Image", "Pre-process", "N4 Bias Correction",
             "Convert to NIFTI for DIPY", "Create Brain Mask", "Tensor Fitting with DIPY"]
    
    output_dir = os.path.join(input_dir, "dti_outputs")
    os.makedirs(output_dir, exist_ok=True)
    
    #dti_path = os.path.join(input_dir, "dti.nii.gz") # new function for using gzip
    bvec_path = os.path.join(input_dir, "bvec.bvec")
    bval_path = os.path.join(input_dir, "bval.bval")
    
    for i, step_name in enumerate(tqdm(steps, desc="Processing DTI Data", unit="step")):
        if step_name == "Ensure Gzipped DTI":
            dti_path = ensure_gzipped_dti(input_dir)

        if step_name == "Convert to MRtrix":
            run_command(f"mrconvert -fslgrad {bvec_path} {bval_path} {dti_path} {output_dir}/nii.mif -force", step_name)
        
        elif step_name == "Denoising":
            denoise_dti(output_dir, dti_path, step_name)
        
        elif step_name == "Extract b0 Image":
            run_command(f"dwiextract -bzero {output_dir}/nii.mif {output_dir}/b0.mif -force", step_name)
        
        elif step_name == "Pre-process":
            eddy_options = "--slm=linear --data_is_shelled"
            run_command(f"dwifslpreproc {output_dir}/nii.mif {output_dir}/dti_denoise_preproc.mif -nocleanup -rpe_none -pe_dir AP -eddy_options \"{eddy_options}\" -force", step_name)
        
        elif step_name == "N4 Bias Correction":
            run_command(f"dwibiascorrect ants {output_dir}/dti_denoise_preproc.mif {output_dir}/dti_denoise_preproc_unbiased.mif -bias {output_dir}/bias.mif -force", step_name)
        
        elif step_name == "Convert to NIFTI for DIPY":
            run_command(f"mrconvert {output_dir}/dti_denoise_preproc_unbiased.mif {output_dir}/dti_denoise_preproc_unbiased.nii -force", step_name)
        
        elif step_name == "Create Brain Mask":
            run_command(f"dwi2mask {output_dir}/dti_denoise_preproc_unbiased.mif {output_dir}/mask.mif -force", step_name)
            run_command(f"mrconvert {output_dir}/mask.mif {output_dir}/mask.nii -force", step_name)
        
        elif step_name == "Tensor Fitting with DIPY":
            fit_tensors_with_dipy(output_dir, bvec_path, bval_path, f"{output_dir}/mask.nii", step_name)


## Path to base path directory here
base_path = "/media/jimric/base_directory"

#### Change names below for new patients, make sure they are in the base_path folder
patient_folders = [
    "patient_name",
    "patient_name",
    "patient_name",
    "patient_name",
    "patient_name",
    "patient_name",
    "patient_name",
]


# Ensure the function and script you're using is defined above this loop
for patient in patient_folders:
    patient_path = os.path.join(base_path, patient, f"{patient}_DTI")
    try:
        print(f"Starting Processing of {patient}")
        process_dti(patient_path)
        print(f"Processing of {patient} completed successfully.")
    except Exception as e:
        print(f"Error while processing {patient}: {str(e)}")

